In [17]:
# We will do the imports required for this notebook here

# numpy provides import array and linear algebra utilities
import numpy as np
print (np.__version__)

# the robotics toolbox provides robotics specific functionality
import roboticstoolbox as rtb

# spatial math provides objects for representing transformations
import spatialmath as sm

1.23.1


## Definição do manipulador ABB IRB 460

Com base no trabalho de Cozmin Cristoiu e Adrian Nicolescu, NEW APPROACH FOR FORWARD KINEMATICS
MODELING OF INDUSTRIAL ROBOTS WITH CLOSED KINEMATIC CHAIN,
as matrizes de transformação referente ao manipulador IRB 460 foram feitas. Só é necessário preencher com os valores não nulos. A função usadda é a `ET`, elementary transformation. Nela definimos cada rotação ou translação que ocorre de um frame para outro.

As matrizes de transformação pode ser visualizadas em
https://docs.google.com/spreadsheets/d/1L_g5gxfoq4cghjRyVMdMbR_i7hNX9XNrxOBWqDRwLb0/edit?usp=sharing

In [54]:
#tamanho dos elos ativos
l1 = 0.2345
l2 = 0.508
l3 = 0.260
l4 = 0.945
l5 = 1.025
l6 = 0.220
l7 = -0.2515

E1 = rtb.ET.tz(l1)         #0_1
E2 = rtb.ET.tx(l3)         #1_2
E3 = rtb.ET.tz(l2)         #1_2
E4 = rtb.ET.tx(l3)         #1_3
E5 = rtb.ET.tz(l2)         #1_3
E6 = rtb.ET.tz(l4)         #2_4
E7 = rtb.ET.tx(-0.400)     #5
E8 = rtb.ET.Ry()           #6
E9 = rtb.ET.tz(l4)         #7
E10 = rtb.ET.tx(l5)         #4_5
E11 = rtb.ET.Ry()           #9
E12 = rtb.ET.tx(-l3)        #9
E13 = rtb.ET.tz(0.437)      #9
E14 = rtb.ET.tx(0.247)      #10
E15 = rtb.ET.tz(0.804)      #10
E16 = rtb.ET.Ry()           #11
E17 = rtb.ET.tx(0.04884)    #12
E18 = rtb.ET.tz(-0.6488)    #12
E19 = rtb.ET.Ry()           #13
E20 = rtb.ET.tx(-0.0350)    #14
E21 = rtb.ET.tz(0.33367)    #14
E22 = rtb.ET.Ry()           #15
E23 = rtb.ET.tx(0.795)      #16
E24 = rtb.ET.tz(-0.193)     #16
E25 = rtb.ET.Ry()           #17
E26 = rtb.ET.Rz()           #18 5_6
E27 = rtb.ET.tx(l6)         #18 5_6
E28 = rtb.ET.tz(l7)         #18 5_6


Como sabemos, o produto vetorial dessas matrizes nos dá a informação que relaciona o primeiro frame 0 até o ultimo frame,
o qual é fixado o órgão terminal. É o chamado `ETS`, elementary transformation sequence.

O artigo citado indica que a matriz de transformada homogêna da base para o órgão terminal é dado por:

`T_base-end_flange = T01*T12*T23*T34*T45*T56`

In [55]:
irb460 = E1 * E2 * E3 * E4 * E5 * E9 * E25 * E26 * E27 *E28

In [56]:
print(irb460)

tz(0.2345) ⊕ tx(0.26) ⊕ tz(0.508) ⊕ tx(0.26) ⊕ tz(0.508) ⊕ tz(0.945) ⊕ Ry(q0) ⊕ Rz(q1) ⊕ tx(0.22) ⊕ tz(-0.2515)


In [64]:
# The ETS class has many usefull properties
# print the number of joints in the irb460 model
print(f"O IRB460 possui {irb460.n} juntas")

O IRB460 possui 2 juntas


Como podemos ver, o RTB apontou que o manipulador possui apenas duas juntas. Isso se deve porque na definição do IRB460 por
meio de `ET`s, e com base na definição para 'T_base-end_flange' que o artigo deduz, apenas duas rotações foram computadas.

In [65]:
# print the number of ETs in the irb460 model
print(f"O IRB460 possui {irb460.m} ETs")

O IRB460 possui 10 ETs


Aqui podemos ver algumas informações que as próprias `ET's` nos formecem. Podemos acessar uma transformação específica
por meio de acesso de index. É dito que quando adicionamos uma ET a uma ETS, é definido um index para a junta. Porém
aqui tive retorno `None`.

In [68]:
# We can access an ET from an ETS as if the ETS were a Python list
print(f"A segunda ET na ETS é {irb460[1]}")

# When a variable ET is added to an ETS, it is assigned a jindex, which is short for joint index
# When given an array of joint coordinates (i.e. joint angles), the ETS will use the jindices of each
# variable ET to correspond with elements of the given joint coordiante array
print(f"A primeira variável de junta possui jindex {irb460[1].jindex}, enquanto a segunda possui jindex {irb460[2].jindex}")

# We can extract all of the variable ETs from the irb460 model as a list
print(f"Todas as variáveis de junta da ETS do IRB460 \n{irb460.joints()}")

A segunda ET na ETS é tx(0.26)
A primeira variável de junta possui jindex None, enquanto a segunda possui jindex None
Todas as variáveis de junta da ETS do IRB460 
[ET.Ry(jindex=0), ET.Rz(jindex=1)]


Aqui calculamos a cinemática direta por três métodos. O primeiro faz uma iteração pelas ET do manipulador. No segundo e 
terceiro método usamos funções próprias do `RTB`. Uma mostra o resultado como um objeto `SE3` e a última, como um `numpy array`.

Todas elas usam o vetor de posição q e q1. Eles contém as coordenadas das juntas para então encontrar a posição e orientação do último frame.

In [62]:
# Using the above methodolgy, we can calculate the forward kinematics of our IRB460 model
# First, we must define the joint coordinates q, to calculate the forward kinematics at
q = np.array([0, -0.3, 0, -2.2, 0, 2, 0.79])
q1 = np.array([2.617993878, 0.6108652382, 0.6981317008, np.pi/2, np.pi/2, 0.5235987756, 0])

# Allocate the resulting forward kinematics array
fk = np.eye(4) #matriz com 1 na diagonal e 0 em todo resto.
fk1 = np.eye(4) #matriz com 1 na diagonal e 0 em todo resto.

# Now we must loop over the ETs in the IRB460 
for et in irb460:
    if et.isjoint:
        # This ET is a variable joint
        # Use the q array to specify the joint angle for the variable ET
        fk = fk @ et.A(q[et.jindex])
    else:
        # This ET is static
        fk = fk @ et.A()

print(sm.SE3(fk))

for et in irb460:
    if et.isjoint:
        # This ET is a variable joint
        # Use the q array to specify the joint angle for the variable ET
        fk1 = fk1 @ et.A(q1[et.jindex])
    else:
        # This ET is static
        fk1 = fk1 @ et.A()

# Pretty print our resulting forward kinematics using an SE3 object
print(sm.SE3(fk1))

   0.9553    0.2955    0         0.7302    
  -0.2955    0.9553    0        -0.06501   
   0         0         1         1.944     
   0         0         0         1         

  -0.7094    0.4967    0.5       0.2382    
   0.5736    0.8192    0         0.1262    
  -0.4096    0.2868   -0.866     2.323     
   0         0         0         1         



In [63]:
# The ETS class has the .fkine method which can calculate the forward kinematics
# The .fkine methods returns an SE3 object
print(f"The fkine method: \n{irb460.fkine(q1)}")

# The .eval method also calculates the forward kinematics but returns an numpy array
# instead of an SE3 object (use this if speed is a priority)
print(f"The eval method: \n{irb460.eval(q)}")

The fkine method: 
  -0.7094    0.4967    0.5       0.2382    
   0.5736    0.8192    0         0.1262    
  -0.4096    0.2868   -0.866     2.323     
   0         0         0         1         

The eval method: 
[[ 0.95533649  0.29552021  0.          0.73017403]
 [-0.29552021  0.95533649  0.         -0.06501445]
 [ 0.          0.          1.          1.944     ]
 [ 0.          0.          0.          1.        ]]
